# Build a Chatbot

## 1. Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. 
This chatbot will be able to have a conversation and remember previous interactions.


Note that this chatbot that we build will only use the language model to have a conversation.
There are several other related concepts that you may be looking for:

- [Conversational RAG](/docs/tutorials/qa_chat_history): Enable a chatbot experience over an external source of data
- [Agents](/docs/tutorials/agents): Build a chatbot that can take actions

This tutorial will cover the basics which will be helpful for those two more advanced topics, but feel free to skip directly to there should you choose.




## 2. Concepts

Here are a few of the high-level components we'll be working with:

- [`Chat Models`](/docs/concepts/#chat-models). The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat Models rather than text LLMs.
- [`Prompt Templates`](/docs/concepts/#prompt-templates), which simplify the process of assembling prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
- [`Chat History`](/docs/concepts/#chat-history), which allows a chatbot to "remember" past interactions and take them into account when responding to followup questions. 
- Debugging and tracing your application using [LangSmith](/docs/concepts/#langsmith)

We'll cover how to fit the above components together to create a powerful conversational chatbot.



## 3. Quickstart



First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env', verbose=True)

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01c4bb1f-0fef-40fd-8e8b-d86c6cf59922-0')

The model on its own does not have any concept of state. For example, if you ask a followup question:

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I don't have access to that information. Would you like me to call you by a specific name?", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3bdad40d-0f69-409c-8d49-1b5675d6f5b6-0')

Let's take a look at the example [LangSmith trace](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question.
This makes for a terrible chatbot experience!

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f8ace71f-5f6b-464f-aaf1-decf44cc9ea6-0')

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally.
So how do we best implement this?

## 4. Message History



We can use a Message History class to wrap our model and make it stateful.
This will keep track of inputs and outputs of the model, and store them in some datastore.
Future interactions will then load those messages and pass them into the chain as part of the input.
Let's see how to use this!

First, let's make sure to install `langchain-community`, as we will be using an integration in there to store message history.

In [5]:
# ! pip install langchain_community

After that, we can import the relevant classes and set up our chain which wraps the model and adds in this message history. A key part here is the function we pass into as the `get_session_history`. This function is expected to take in a `session_id` and return a Message History object. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain (we'll show how to do that.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

We now need to create a `config` that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a `session_id`. This should look like:

In [7]:
config = {"configurable": {"session_id": "abc2"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run 7b030c9e-03dc-4a78-89d6-105a6254081c not found for run 1501edc6-2503-4846-9942-d1e8f5171a4e. Treating as a root run.


'Hello Bob! How can I assist you today?'

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 608aef5d-6d86-4070-9bbb-c5e13f5c4f71 not found for run 8b58ad0e-a452-4d56-b98a-21e393652a37. Treating as a root run.


'Your name is Bob.'

Great! Our chatbot now remembers things about us. If we change the config to reference a different `session_id`, we can see that it starts the conversation fresh.

In [10]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run a60bcb1d-adb6-46dc-986a-cb4d5991b173 not found for run 839cd130-7d82-4d55-9662-8c54afcdd4d2. Treating as a root run.


"I'm sorry, but I do not have access to that information."

However, we can always go back to the original conversation (since we are persisting it in a database)

In [11]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run b450791e-f5e6-4a44-8968-f6a71830b00c not found for run f82c3bf5-c698-4fd6-aeb8-fb86a1d59c2c. Treating as a root run.


'Your name is Bob.'

This is how we can support a chatbot having conversations with many users!

Right now, all we've done is add a simple persistence layer around the model. We can start to make the more complicated and personalized by adding in a prompt template.

## 5. Prompt templates



Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

First, let's add in a system message. To do this, we will create a ChatPromptTemplate. We will utilize `MessagesPlaceholder` to pass all the messages in.

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Note that this slightly changes the input type - rather than pass in a list of messages, we are now passing in a dictionary with a `messages` key where that contains a list of messages.

In [13]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello, Bob! How can I assist you today?'

We can now wrap this in the same Messages History object as before

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. Answer all questions to the best of your ability.')), MessagesPlaceholder(variable_name='messages')])
  | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb2c3327580>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb2c3337160>, openai_api_key=SecretStr('**********'), openai_proxy=''),

In [15]:
config = {"configurable": {"session_id": "abc5"}}

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

Parent run a2b6b386-4ba7-48b6-8877-ac39149d4530 not found for run a586ea35-6c3f-4347-b63d-90ddf7acbc10. Treating as a root run.


'Hello, Jim! How can I assist you today?'

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 80c90260-35be-48aa-ac5b-6d3fade423a1 not found for run ffe775ee-8343-4bfe-a8a1-f9254436b7dc. Treating as a root run.


'Your name is Jim.'

Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Note that we have added a new `language` input to the prompt. We can now invoke the chain and pass in a language of our choice.

In [19]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Chinese"}
)

response.content

'你好！很高兴认识你，我是你的助手。有什么可以帮助你的吗？'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [21]:
config = {"configurable": {"session_id": "abc11"}}

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "粤语"},
    config=config,
)

response.content

Parent run 2c733fd6-a4af-4b0b-9c99-e0c1b786ddef not found for run df0f0efc-2137-416f-b15d-34cb2e595b7e. Treating as a root run.


'嗨！你好，Todd！有什么我可以帮到你嘅吗？'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "粤语"},
    config=config,
)

response.content

Parent run c1dd3e03-b190-456d-a0e4-f75dd89d62d3 not found for run d446e248-6e94-4507-a668-c5cfecd86922. Treating as a root run.


'你嘅名叫 Todd。有其他问题需要帮忙吗？'

To help you understand what's happening internally, check out [this LangSmith trace](https://smith.langchain.com/public/f48fabb6-6502-43ec-8242-afc352b769ed/r)

## 6. Managing Conversation History



One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

**Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.**

We can do this by adding a simple step in front of the prompt that modifies the `messages` key appropriately, and then wrap that new chain in the Message History class. First, let's define a function that will modify the messages passed in. Let's make it so that it selects the `k` most recent messages. We can then create a new chain by adding that at the start.

In [24]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

Let's now try it out! If we create a list of messages more than 10 messages long, we can see what it no longer remembers information in the early messages.

In [25]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to your personal information."

But if we ask about information that is within the last ten messages, it still remembers it

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'Your favorite ice cream flavor is vanilla.'

Let's now wrap this in the Message History

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 875b05a2-87b1-4a33-b8e2-ca87398a60a0 not found for run d21955c3-5136-40df-9131-336d640477d7. Treating as a root run.


"I'm sorry, I don't have access to personal information."

There's now two new messages in the chat history. This means that even more information that used to be accessible in our conversation history is no longer available!

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run cbc06391-b966-43a4-bcdd-d7fe659cf21a not found for run 0cb4b891-84e8-4dfb-a521-de6ef9a02cba. Treating as a root run.


"I'm sorry, I don't have that information."

If you take a look at LangSmith, you can see exactly what is happening under the hood in the [LangSmith trace](https://smith.langchain.com/public/fa6b00da-bcd8-4c1c-a799-6b32a3d62964/r)

## 7. Streaming



Now we've got a function chatbot. However, one *really* important UX consideration for chatbot application is streaming. LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most application do is stream back each token as it is generated. This allows the user to see progress.

It's actually super easy to do this!

All chains expose a `.stream` method, and ones that use message history are no different. We can simply use that method to get back a streaming response.

In [31]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run c91884c3-3eae-4604-886d-ad3e46769cb4 not found for run 86700249-0ec1-4672-883a-38ed27df27f2. Treating as a root run.


|Sure|,| Todd|!| Here|'s| a| joke| for| you|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?
|Because| it| was| two| tired|!||

## 8. Next Steps

Now that you understand the basics of how to create a chatbot in LangChain, some more advanced tutorials you may be interested in are:

- [Conversational RAG](/docs/tutorials/qa_chat_history): Enable a chatbot experience over an external source of data
- [Agents](/docs/tutorials/agents): Build a chatbot that can take actions

If you want to dive deeper on specifics, some things worth checking out are:

- [Streaming](/docs/how_to/streaming): streaming is *crucial* for chat applications
- [How to add message history](/docs/how_to/message_history): for a deeper dive into all things related to message history